In [1]:
from keras.layers import  Dense
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split

In [3]:
from keras.preprocessing import image_dataset_from_directory

In [4]:
ruta_dataset_homen = "/prj/posgrad/samuelrt/Documentos/Doutorado/images/homen"
ruta_dataset_mulher = "/prj/posgrad/samuelrt/Documentos/Doutorado/images/mulher"

# Cargar las imágenes en conjuntos de entrenamiento y validación
train_ds = image_dataset_from_directory(
    ruta_dataset,
    validation_split=0.2,  # 80% entrenamiento, 20% validación
    subset="training",
    seed=123,
    image_size=(224, 224),  # Tamaño estándar para modelos preentrenados
    batch_size=32
)

val_ds = image_dataset_from_directory(
    ruta_dataset,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Verificar clases cargadas
clases = train_ds.class_names
print("Clases detectadas:", clases)

NameError: name 'ruta_dataset' is not defined